In [ ]:
#Prueba de conexión para el sensor de sonido 2 utilizando la libreria request para protocolos HTTP
import requests
url = 'https://industrial.api.ubidots.com/api/v2.0/devices/5f68c56265a24d243a492036/variables/5f68c60a1d84727f19bd0ce9/?token=BBFF-XHVn6EsgdbgsJHa5YvzxUYGA5quWj7'
req = requests.get(url, params=None)
req.json()

{'url': 'https://industrial.api.ubidots.com/api/v2.0/variables/5f68c60a1d84727f19bd0ce9',
 'id': '5f68c60a1d84727f19bd0ce9',
 'label': 'illuminance',
 'name': 'Iluminancia',
 'description': '',
 'device': {'url': 'https://industrial.api.ubidots.com/api/v2.0/devices/5f68c56265a24d243a492036',
  'id': '5f68c56265a24d243a492036',
  'label': 'nxsensor2_3ffcaa_rk210_01_eds2000',
  'name': 'NxS2 luxes y presencia (01)',
  'createdAt': '2020-09-21T15:23:14.667000Z'},
 'tags': [],
 'properties': {'_current_organization_id': 62309,
  '_previous_variable_label': 'illuminance',
  '_previous_organization_id': 62309},
 'type': 'raw',
 'unit': 'klx',
 'syntheticExpression': '',
 'createdAt': '2020-09-21T15:26:02.436000Z',
 'lastValue': {'value': 159.2,
  'timestamp': 1700202025049,
  'context': {},
  'created_at': 1700202325880},
 'lastActivity': 1700202325885,
 'valuesUrl': 'https://industrial.api.ubidots.com/api/v1.6/variables/5f68c60a1d84727f19bd0ce9/values'}

# Sensor Sonido 2

In [ ]:
import requests

# URL and token setup
url = 'https://industrial.api.ubidots.com/api/v1.6/variables/5f68c60a1d84727f19bd0ce9/values/'
token = 'BBFF-XHVn6EsgdbgsJHa5YvzxUYGA5quWj7'  # Replace with your actual token

# Headers
headers = {'X-Auth-Token': token}

# Send GET request to Ubidots API
response = requests.get(url, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    data = response.json()

    # Initialize lists to store the data
    values_list = []
    timestamps_list = []

    # Iterate over the results and populate the lists
    for entry in data.get('results', []):
        values_list.append(entry.get('value'))
        timestamps_list.append(entry.get('timestamp'))

    # Print the lists
    print("Values:", values_list)
    print("Timestamps:", timestamps_list)
else:
    print("Error:", response.status_code)

Values: [158.6, 158.8, 159.2, 159.2, 159.2, 159.2, 159.2, 159.2, 159.2, 159.2, 159.2, 159.2, 159.2, 159.2, 158.6, 158.6, 158.6, 158.6, 158.6, 158.6, 158.6, 158.6, 158.6, 158.6, 158.6, 158.8, 158.8, 158.8, 158.8, 158.8, 158.8, 158.8, 158.8, 158.8, 158.8, 158.8, 159.0, 159.2, 159.0, 158.6, 158.6, 158.6, 158.8, 158.8, 158.8, 158.6, 158.6, 158.6, 159.2, 158.2, 159.2, 158.8, 158.8, 158.8, 158.8, 158.8, 159.2, 159.2, 159.2, 159.2, 159.2, 159.2, 159.2, 159.2, 159.2, 159.2, 158.8, 158.8, 158.8, 159.2, 159.2, 159.2, 159.0, 159.0, 159.0, 158.2, 159.2, 159.6, 158.4, 159.4, 158.6, 159.2, 159.2, 159.2, 159.2, 159.2, 159.2, 159.2, 158.6, 158.6, 158.6, 158.6, 158.6, 158.6, 158.6, 158.6, 158.6, 158.6, 158.6, 158.6]
Timestamps: [1700211373564, 1700211073564, 1700210773564, 1700210630632, 1700210616542, 1700210610225, 1700210600518, 1700210584470, 1700210572428, 1700210572026, 1700210547114, 1700210515387, 1700210483244, 1700210469881, 1700210330632, 1700210316542, 1700210310225, 1700210300518, 17002102

In [ ]:
import requests
import time

# URL and token setup
url = 'https://industrial.api.ubidots.com/api/v1.6/variables/5f68c60a1d84727f19bd0ce9/values/'
token = 'BBFF-XHVn6EsgdbgsJHa5YvzxUYGA5quWj7'  # Replace with your actual token

# Headers for Ubidots
headers_ubidots = {'X-Auth-Token': token}

# Send GET request to Ubidots API
response_ubidots = requests.get(url, headers=headers_ubidots)

# Check if the request was successful
if response_ubidots.status_code == 200:
    data_ubidots = response_ubidots.json()

    # Orion Context Broker endpoint
    orion_url = 'http://44.205.186.77:1026/v2/entities/urn:ngsi-ld:Sensor:NxS2:Sonido:02/attrs'

    # Headers for Orion
    headers_orion = {'Content-Type': 'application/json'}

    # Iterate over the results and send each data point to Fiware
    for entry in data_ubidots.get('results', []):
        value = entry.get('value')
        timestamp = entry.get('timestamp')  # Assuming timestamp is in milliseconds since epoch

        # JSON payload for the PATCH request
        payload = {
            "Sonido": {
                "value": value,
                "type": "Float"
            },
            "ObservedAt": {
                "type": "Timestamp",
                "value": timestamp,
                "metadata": {}
            }
        }

        # Send PATCH request to Orion Context Broker
        response_orion = requests.patch(orion_url, json=payload, headers=headers_orion)
        time.sleep(0.8)
        # Check the response from Orion
        #if response_orion.status_code != 204:
            #print(f"Failed to update entity: {response_orion.status_code}")
            #print(f"Response: {response_orion.json()}")
            #time.sleep(0.01)

            #break  # If one request fails, break out of the loop
else:
    print("Error fetching data from Ubidots:", response_ubidots.status_code)

# Sensor Humedad y Temperatura 1

Humedad

In [ ]:
import requests
url2 = 'https://industrial.api.ubidots.com/api/v2.0/devices/5f68c56d65a24d243a492040/variables/5f68c5fc1d84727e16a68859/?token=BBFF-BcXDn9Awb6lYDtq5ZEBhIMnwW5CEH6'
url3 = 'https://industrial.api.ubidots.com/api/v2.0/devices/5f68c56d65a24d243a492040/variables/5f68c6021d84727db456c2cb/?token=BBFF-BcXDn9Awb6lYDtq5ZEBhIMnwW5CEH6'
req2 = requests.get(url2, params=None)
req2.json()

{'url': 'https://industrial.api.ubidots.com/api/v2.0/variables/5f68c5fc1d84727e16a68859',
 'id': '5f68c5fc1d84727e16a68859',
 'label': 'humidity',
 'name': 'Humedad',
 'description': '',
 'device': {'url': 'https://industrial.api.ubidots.com/api/v2.0/devices/5f68c56d65a24d243a492040',
  'id': '5f68c56d65a24d243a492040',
  'label': 'nxsensor2_3fda76_sht10',
  'name': 'NxS2 humedad y temperatura ambiente (01)',
  'createdAt': '2020-09-21T15:23:25.431000Z'},
 'tags': [],
 'properties': {'_current_organization_id': 62309,
  '_previous_variable_label': 'humidity',
  '_previous_organization_id': 62309},
 'type': 'raw',
 'unit': '%',
 'syntheticExpression': '',
 'createdAt': '2020-09-21T15:25:48.180000Z',
 'lastValue': {'value': 60.7,
  'timestamp': 1699907218863,
  'context': {},
  'created_at': 1699907219612},
 'lastActivity': 1699907219617,
 'valuesUrl': 'https://industrial.api.ubidots.com/api/v1.6/variables/5f68c5fc1d84727e16a68859/values'}

In [ ]:
req3 = requests.get(url3, params=None)
req3.json()

{'url': 'https://industrial.api.ubidots.com/api/v2.0/variables/5f68c6021d84727db456c2cb',
 'id': '5f68c6021d84727db456c2cb',
 'label': 'temperature',
 'name': 'Temperature',
 'description': '',
 'device': {'url': 'https://industrial.api.ubidots.com/api/v2.0/devices/5f68c56d65a24d243a492040',
  'id': '5f68c56d65a24d243a492040',
  'label': 'nxsensor2_3fda76_sht10',
  'name': 'NxS2 humedad y temperatura ambiente (01)',
  'createdAt': '2020-09-21T15:23:25.431000Z'},
 'tags': [],
 'properties': {'_current_organization_id': 62309,
  '_previous_variable_label': 'temperature',
  '_previous_organization_id': 62309},
 'type': 'raw',
 'unit': '°C',
 'syntheticExpression': '',
 'createdAt': '2020-09-21T15:25:54.863000Z',
 'lastValue': {'value': 27.3,
  'timestamp': 1699907218863,
  'context': {},
  'created_at': 1699907219612},
 'lastActivity': 1699907219619,
 'valuesUrl': 'https://industrial.api.ubidots.com/api/v1.6/variables/5f68c6021d84727db456c2cb/values'}

In [ ]:
import requests

# URL and token setup
url = 'https://industrial.api.ubidots.com/api/v1.6/variables/5f68c5fc1d84727e16a68859/values/'
token = 'BBFF-BcXDn9Awb6lYDtq5ZEBhIMnwW5CEH6'  # Replace with your actual token

# Headers
headers = {'X-Auth-Token': token}

# Send GET request to Ubidots API
response = requests.get(url, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    data = response.json()

    # Initialize lists to store the data
    values_list = []
    timestamps_list = []

    # Iterate over the results and populate the lists
    for entry in data.get('results', []):
        values_list.append(entry.get('value'))
        timestamps_list.append(entry.get('timestamp'))

    # Print the lists
    print("Values:", values_list)
    print("Timestamps:", timestamps_list)
else:
    print("Error:", response.status_code)

Values: [60.7, 60.7, 60.7, 59.5, 59.5, 59.5, 60.2, 60.2, 60.2, 59.4, 58.1, 57.6, 58.5, 58.5, 59.0, 58.1, 58.1, 59.0, 59.0, 60.2, 60.2, 59.8, 59.4, 58.8, 58.1, 58.1, 56.2, 56.2, 55.3, 55.3, 54.0, 52.3, 52.0, 51.8, 51.6, 51.0, 50.0, 50.0, 50.2, 50.2, 50.0, 50.0, 49.5, 49.6, 50.0, 49.7, 49.7, 49.7, 49.4, 49.4, 49.4, 50.3, 50.3, 50.3, 51.3, 51.6, 51.3, 51.1, 89.0, 90.0, 90.0, 90.3, 90.3, 90.1, 90.1, 90.8, 90.8, 90.8, 90.6, 90.6, 90.6, 90.9, 90.9, 90.9, 90.6, 90.7, 90.7, 92.1, 92.0, 91.8, 92.0, 92.0, 92.0, 92.0, 92.0, 92.4, 92.4, 92.4, 92.4, 92.4, 92.3, 92.3, 92.3, 92.3, 92.3, 92.1, 92.1, 92.1, 92.1, 92.1]
Timestamps: [1699907218863, 1699907214482, 1699907207191, 1699906918863, 1699906914482, 1699906907191, 1699906618863, 1699906614482, 1699906607191, 1699906304667, 1699906004667, 1699905704667, 1699905395069, 1699905095069, 1699904795069, 1699904495627, 1699904493426, 1699904195627, 1699904193426, 1699903895627, 1699903893426, 1699903585458, 1699903285458, 1699902985458, 1699902686172, 169

In [ ]:
len(values_list)

100

In [ ]:
import requests
import time

# URL and token setup
url = 'https://industrial.api.ubidots.com/api/v1.6/variables/5f68c5fc1d84727e16a68859/values/'
token = 'BBFF-BcXDn9Awb6lYDtq5ZEBhIMnwW5CEH6'  # Replace with your actual token

# Headers for Ubidots
headers_ubidots = {'X-Auth-Token': token}

# Send GET request to Ubidots API
response_ubidots = requests.get(url, headers=headers_ubidots)

# Check if the request was successful
if response_ubidots.status_code == 200:
    data_ubidots = response_ubidots.json()

    # Orion Context Broker endpoint
    orion_url = 'http://44.205.186.77:1026/v2/entities/urn:ngsi-ld:Sensor:NxS2:HumedadTemperaturaAmbiente:01/attrs'

    # Headers for Orion
    headers_orion = {'Content-Type': 'application/json'}

    # Iterate over the results and send each data point to Fiware
    for entry in data_ubidots.get('results', []):
        value = entry.get('value')
        timestamp = entry.get('timestamp')  # Assuming timestamp is in milliseconds since epoch

        # JSON payload for the PATCH request
        payload = {
            "Humedad": {
                "value": value,
                "type": "Float"
            },
            "ObservedAt": {
                "type": "Timestamp",
                "value": timestamp,
                "metadata": {}
            }
        }

        # Send PATCH request to Orion Context Broker
        response_orion = requests.patch(orion_url, json=payload, headers=headers_orion)
        time.sleep(1)
        # Check the response from Orion
        #if response_orion.status_code != 204:
            #print(f"Failed to update entity: {response_orion.status_code}")
            #print(f"Response: {response_orion.json()}")
            #time.sleep(0.01)

            #break  # If one request fails, break out of the loop
else:
    print("Error fetching data from Ubidots:", response_ubidots.status_code)

Temperatura

In [ ]:
import requests

# URL and token setup
url = 'https://industrial.api.ubidots.com/api/v1.6/variables/5f68c6021d84727db456c2cb/values/'
token = 'BBFF-BcXDn9Awb6lYDtq5ZEBhIMnwW5CEH6'  # Replace with your actual token

# Headers
headers = {'X-Auth-Token': token}

# Send GET request to Ubidots API
response = requests.get(url, headers=headers)
response.json()

{'count': True,
 'next': 'https://industrial.api.ubidots.com/api/v1.6/variables/5f68c6021d84727db456c2cb/values/?page=2',
 'previous': None,
 'results': [{'timestamp': 1699907218863,
   'created_at': 1699907219612,
   'value': 27.3,
   'context': {}},
  {'timestamp': 1699907214482,
   'created_at': 1699907215907,
   'value': 27.3,
   'context': {}},
  {'timestamp': 1699907207191,
   'created_at': 1699907209471,
   'value': 27.3,
   'context': {}},
  {'timestamp': 1699906918863,
   'created_at': 1699907219612,
   'value': 27.5,
   'context': {}},
  {'timestamp': 1699906914482,
   'created_at': 1699907215907,
   'value': 27.5,
   'context': {}},
  {'timestamp': 1699906907191,
   'created_at': 1699907209471,
   'value': 27.5,
   'context': {}},
  {'timestamp': 1699906618863,
   'created_at': 1699907219612,
   'value': 27.6,
   'context': {}},
  {'timestamp': 1699906614482,
   'created_at': 1699907215907,
   'value': 27.6,
   'context': {}},
  {'timestamp': 1699906607191,
   'created_at': 

In [ ]:
# Check if the request was successful
if response.status_code == 200:
    data = response.json()

    # Initialize lists to store the data
    values_list = []
    timestamps_list = []

    # Iterate over the results and populate the lists
    for entry in data.get('results', []):
        values_list.append(entry.get('value'))
        timestamps_list.append(entry.get('timestamp'))

    # Print the lists
    print("Values:", values_list)
    print("Timestamps:", timestamps_list)
else:
    print("Error:", response.status_code)

In [ ]:
len(values_list)

100

In [ ]:
import requests
import time

# URL and token setup
url = 'https://industrial.api.ubidots.com/api/v1.6/variables/5f68c6021d84727db456c2cb/values/'
token = 'BBFF-BcXDn9Awb6lYDtq5ZEBhIMnwW5CEH6'  # Replace with your actual token

# Headers for Ubidots
headers_ubidots = {'X-Auth-Token': token}

# Send GET request to Ubidots API
response_ubidots = requests.get(url, headers=headers_ubidots)

# Check if the request was successful
if response_ubidots.status_code == 200:
    data_ubidots = response_ubidots.json()

    # Orion Context Broker endpoint
    orion_url = 'http://44.205.186.77:1026/v2/entities/urn:ngsi-ld:Sensor:NxS2:HumedadTemperaturaAmbiente:01/attrs'

    # Headers for Orion
    headers_orion = {'Content-Type': 'application/json'}

    # Iterate over the results and send each data point to Fiware
    for entry in data_ubidots.get('results', []):
        value = entry.get('value')
        timestamp = entry.get('timestamp')  # Assuming timestamp is in milliseconds since epoch

        # JSON payload for the PATCH request
        payload = {
            "Temperatura": {
                "value": value,
                "type": "Float"
            },
            "ObservedAt": {
                "type": "Timestamp",
                "value": timestamp,
                "metadata": {}
            }
        }

        # Send PATCH request to Orion Context Broker
        response_orion = requests.patch(orion_url, json=payload, headers=headers_orion)
        time.sleep(1)
        # Check the response from Orion
        #if response_orion.status_code != 204:
            #print(f"Failed to update entity: {response_orion.status_code}")
            #print(f"Response: {response_orion.json()}")
            #time.sleep(0.01)

            #break  # If one request fails, break out of the loop
else:
    print("Error fetching data from Ubidots:", response_ubidots.status_code)

# Sensor Temperatura y Humedad 2

Humedad

In [ ]:
import requests
url4 = 'https://industrial.api.ubidots.com/api/v2.0/devices/5f6de6d365a24d0ebb1d82cc/variables/5f6e015a1d84721446dc90bb/?token=BBFF-xvQTfP0NoR7rmFfZGf3NNxVNR4jKZQ'
url5 = 'https://industrial.api.ubidots.com/api/v2.0/devices/5f6de6d365a24d0ebb1d82cc/variables/5f6e01551d8472144b1044f6/?token=BBFF-xvQTfP0NoR7rmFfZGf3NNxVNR4jKZQ'
req4 = requests.get(url4, params=None)
req4.json()

{'url': 'https://industrial.api.ubidots.com/api/v2.0/variables/5f6e015a1d84721446dc90bb',
 'id': '5f6e015a1d84721446dc90bb',
 'label': 'humidity',
 'name': 'Humedad',
 'description': '',
 'device': {'url': 'https://industrial.api.ubidots.com/api/v2.0/devices/5f6de6d365a24d0ebb1d82cc',
  'id': '5f6de6d365a24d0ebb1d82cc',
  'label': 'nxsensor2_3fd3ec_sht10',
  'name': 'NxS2 humedad y temperatura ambiente (02)',
  'createdAt': '2020-09-25T12:47:15.940000Z'},
 'tags': [],
 'properties': {'_current_organization_id': 62309,
  '_previous_variable_label': 'humidity',
  '_previous_organization_id': 62309},
 'type': 'raw',
 'unit': '%',
 'syntheticExpression': '',
 'createdAt': '2020-09-25T14:40:26.729000Z',
 'lastValue': {'value': 85.9,
  'timestamp': 1700214896319,
  'context': {},
  'created_at': 1700214896656},
 'lastActivity': 1700214896661,
 'valuesUrl': 'https://industrial.api.ubidots.com/api/v1.6/variables/5f6e015a1d84721446dc90bb/values'}

In [ ]:
req5 = requests.get(url5, params=None)
req5.json()

{'url': 'https://industrial.api.ubidots.com/api/v2.0/variables/5f6e01551d8472144b1044f6',
 'id': '5f6e01551d8472144b1044f6',
 'label': 'temperature',
 'name': 'Temperature',
 'description': '',
 'device': {'url': 'https://industrial.api.ubidots.com/api/v2.0/devices/5f6de6d365a24d0ebb1d82cc',
  'id': '5f6de6d365a24d0ebb1d82cc',
  'label': 'nxsensor2_3fd3ec_sht10',
  'name': 'NxS2 humedad y temperatura ambiente (02)',
  'createdAt': '2020-09-25T12:47:15.940000Z'},
 'tags': [],
 'properties': {'_current_organization_id': 62309,
  '_previous_variable_label': 'temperature',
  '_previous_organization_id': 62309},
 'type': 'raw',
 'unit': '°C',
 'syntheticExpression': '',
 'createdAt': '2020-09-25T14:40:21.154000Z',
 'lastValue': {'value': 20.6,
  'timestamp': 1700214896319,
  'context': {},
  'created_at': 1700214896656},
 'lastActivity': 1700214896663,
 'valuesUrl': 'https://industrial.api.ubidots.com/api/v1.6/variables/5f6e01551d8472144b1044f6/values'}

In [ ]:
import requests

# URL and token setup
url = 'https://industrial.api.ubidots.com/api/v1.6/variables/5f6e015a1d84721446dc90bb/values/'
token = 'BBFF-xvQTfP0NoR7rmFfZGf3NNxVNR4jKZQ'  # Replace with your actual token

# Headers
headers = {'X-Auth-Token': token}

# Send GET request to Ubidots API
response = requests.get(url, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    data = response.json()

    # Initialize lists to store the data
    values_list = []
    timestamps_list = []

    # Iterate over the results and populate the lists
    for entry in data.get('results', []):
        values_list.append(entry.get('value'))
        timestamps_list.append(entry.get('timestamp'))

    # Print the lists
    print("Values:", values_list)
    print("Timestamps:", timestamps_list)
else:
    print("Error:", response.status_code)

Values: [85.9, 85.9, 86.0, 86.0, 85.7, 85.7, 86.2, 85.1, 85.6, 85.7, 85.4, 85.4, 86.0, 86.0, 86.0, 86.0, 86.0, 86.6, 86.6, 86.6, 86.6, 86.6, 86.4, 86.4, 86.4, 86.4, 86.4, 87.0, 86.7, 85.9, 85.7, 85.7, 85.7, 85.7, 85.7, 85.7, 85.9, 85.9, 85.9, 85.9, 85.9, 85.9, 85.8, 85.8, 85.8, 85.8, 85.8, 85.8, 84.7, 84.6, 84.7, 84.4, 84.4, 84.2, 84.2, 84.4, 84.4, 84.1, 84.1, 84.1, 84.1, 84.1, 84.2, 84.2, 84.2, 84.2, 84.2, 85.0, 85.0, 85.0, 85.0, 85.0, 85.3, 85.7, 85.8, 85.7, 85.5, 85.3, 85.3, 85.4, 85.2, 84.8, 83.9, 84.5, 84.3, 84.0, 83.6, 83.6, 83.8, 84.4, 84.4, 84.7, 84.9, 86.1, 86.3, 87.0, 87.0, 86.7, 86.6, 86.5]
Timestamps: [1700214896319, 1700214889308, 1700214596319, 1700214589308, 1700214296319, 1700214289308, 1700213984369, 1700213684369, 1700213384369, 1700213079112, 1700212779112, 1700212479112, 1700212322280, 1700212303713, 1700212295533, 1700212252601, 1700212184701, 1700212022280, 1700212003713, 1700211995533, 1700211952601, 1700211884701, 1700211722280, 1700211703713, 1700211695533, 170

In [ ]:
len(values_list)

100

In [ ]:
import requests
import time

# URL and token setup
url = 'https://industrial.api.ubidots.com/api/v1.6/variables/5f6e015a1d84721446dc90bb/values/'
token = 'BBFF-xvQTfP0NoR7rmFfZGf3NNxVNR4jKZQ'  # Replace with your actual token

# Headers for Ubidots
headers_ubidots = {'X-Auth-Token': token}

# Send GET request to Ubidots API
response_ubidots = requests.get(url, headers=headers_ubidots)

# Check if the request was successful
if response_ubidots.status_code == 200:
    data_ubidots = response_ubidots.json()

    # Orion Context Broker endpoint
    orion_url = 'http://44.205.186.77:1026/v2/entities/urn:ngsi-ld:Sensor:NxS2:HumedadTemperaturaAmbiente:05/attrs'

    # Headers for Orion
    headers_orion = {'Content-Type': 'application/json'}

    # Iterate over the results and send each data point to Fiware
    for entry in data_ubidots.get('results', []):
        value = entry.get('value')
        timestamp = entry.get('timestamp')  # Assuming timestamp is in milliseconds since epoch

        # JSON payload for the PATCH request
        payload = {
            "Humedad": {
                "value": value,
                "type": "Float"
            },
            "ObservedAt": {
                "type": "Timestamp",
                "value": timestamp,
                "metadata": {}
            }
        }

        # Send PATCH request to Orion Context Broker
        response_orion = requests.patch(orion_url, json=payload, headers=headers_orion)
        time.sleep(1)
        # Check the response from Orion
        #if response_orion.status_code != 204:
            #print(f"Failed to update entity: {response_orion.status_code}")
            #print(f"Response: {response_orion.json()}")
            #time.sleep(0.01)

            #break  # If one request fails, break out of the loop
else:
    print("Error fetching data from Ubidots:", response_ubidots.status_code)

Temperatura

In [ ]:
import requests

# URL and token setup
url = 'https://industrial.api.ubidots.com/api/v1.6/variables/5f6e01551d8472144b1044f6/values/'
token = 'BBFF-xvQTfP0NoR7rmFfZGf3NNxVNR4jKZQ'  # Replace with your actual token

# Headers
headers = {'X-Auth-Token': token}

# Send GET request to Ubidots API
response = requests.get(url, headers=headers)
response.json()

{'count': True,
 'next': 'https://industrial.api.ubidots.com/api/v1.6/variables/5f6e01551d8472144b1044f6/values/?page=2',
 'previous': None,
 'results': [{'timestamp': 1700215945158,
   'created_at': 1700215947562,
   'value': 20.6,
   'context': {}},
  {'timestamp': 1700215866743,
   'created_at': 1700215874601,
   'value': 20.6,
   'context': {}},
  {'timestamp': 1700215804188,
   'created_at': 1700215804611,
   'value': 20.6,
   'context': {}},
  {'timestamp': 1700215645158,
   'created_at': 1700215947562,
   'value': 20.5,
   'context': {}},
  {'timestamp': 1700215566743,
   'created_at': 1700215874601,
   'value': 20.5,
   'context': {}},
  {'timestamp': 1700215504188,
   'created_at': 1700215804611,
   'value': 20.5,
   'context': {}},
  {'timestamp': 1700215345158,
   'created_at': 1700215947562,
   'value': 20.6,
   'context': {}},
  {'timestamp': 1700215266743,
   'created_at': 1700215874601,
   'value': 20.6,
   'context': {}},
  {'timestamp': 1700215204188,
   'created_at': 

In [ ]:
# Check if the request was successful
if response.status_code == 200:
    data = response.json()

    # Initialize lists to store the data
    values_list = []
    timestamps_list = []

    # Iterate over the results and populate the lists
    for entry in data.get('results', []):
        values_list.append(entry.get('value'))
        timestamps_list.append(entry.get('timestamp'))

    # Print the lists
    print("Values:", values_list)
    print("Timestamps:", timestamps_list)
else:
    print("Error:", response.status_code)

Values: [20.6, 20.6, 20.6, 20.5, 20.5, 20.5, 20.6, 20.6, 20.6, 20.6, 20.6, 20.7, 20.7, 20.7, 20.7, 20.7, 20.7, 20.7, 20.6, 20.7, 20.7, 20.6, 20.6, 20.6, 20.6, 20.6, 20.6, 20.6, 20.6, 20.6, 20.6, 20.6, 20.6, 20.6, 20.6, 20.6, 20.6, 20.7, 20.9, 20.8, 20.8, 20.8, 20.8, 20.8, 20.8, 21.0, 21.0, 21.0, 21.0, 21.0, 21.0, 21.0, 21.0, 21.0, 21.0, 21.0, 21.0, 21.0, 21.1, 21.1, 21.1, 21.1, 21.0, 21.0, 20.9, 20.9, 21.0, 21.0, 21.0, 21.0, 21.0, 21.0, 21.0, 21.0, 21.0, 21.0, 21.0, 21.0, 21.0, 21.0, 21.0, 21.0, 21.0, 21.0, 21.0, 21.0, 21.1, 21.1, 21.0, 20.9, 20.9, 21.0, 21.0, 21.0, 21.0, 20.9, 21.0, 20.9, 20.9, 21.0]
Timestamps: [1700215945158, 1700215866743, 1700215804188, 1700215645158, 1700215566743, 1700215504188, 1700215345158, 1700215266743, 1700215204188, 1700214896319, 1700214889308, 1700214596319, 1700214589308, 1700214296319, 1700214289308, 1700213984369, 1700213684369, 1700213384369, 1700213079112, 1700212779112, 1700212479112, 1700212322280, 1700212303713, 1700212295533, 1700212252601, 170

In [ ]:
len(values_list)

100

In [ ]:
import requests
import time

# URL and token setup
url = 'https://industrial.api.ubidots.com/api/v1.6/variables/5f6e01551d8472144b1044f6/values/'
token = 'BBFF-xvQTfP0NoR7rmFfZGf3NNxVNR4jKZQ'  # Replace with your actual token

# Headers for Ubidots
headers_ubidots = {'X-Auth-Token': token}

# Send GET request to Ubidots API
response_ubidots = requests.get(url, headers=headers_ubidots)

# Check if the request was successful
if response_ubidots.status_code == 200:
    data_ubidots = response_ubidots.json()

    # Orion Context Broker endpoint
    orion_url = 'http://44.205.186.77:1026/v2/entities/urn:ngsi-ld:Sensor:NxS2:HumedadTemperaturaAmbiente:05/attrs'

    # Headers for Orion
    headers_orion = {'Content-Type': 'application/json'}

    # Iterate over the results and send each data point to Fiware
    for entry in data_ubidots.get('results', []):
        value = entry.get('value')
        timestamp = entry.get('timestamp')  # Assuming timestamp is in milliseconds since epoch

        # JSON payload for the PATCH request
        payload = {
            "Temperatura": {
                "value": value,
                "type": "Float"
            },
            "ObservedAt": {
                "type": "Timestamp",
                "value": timestamp,
                "metadata": {}
            }
        }

        # Send PATCH request to Orion Context Broker
        response_orion = requests.patch(orion_url, json=payload, headers=headers_orion)
        time.sleep(1)
        # Check the response from Orion
        #if response_orion.status_code != 204:
            #print(f"Failed to update entity: {response_orion.status_code}")
            #print(f"Response: {response_orion.json()}")
            #time.sleep(0.01)

            #break  # If one request fails, break out of the loop
else:
    print("Error fetching data from Ubidots:", response_ubidots.status_code)